In [51]:
import pandas as pd

In [52]:
import plotly.express as px

In [53]:
def fix_time(x):
    if not '+' in x:
        return 0
    else:  
        s = x.split(' ')
        if 'min' in x:
            return 60*int(s[1]) + int(s[-2])
        else:
            return int(s[-2])

In [54]:
df_giro = pd.read_csv('./data/giro.csv', sep=';', names=['position','name','team','time'])

In [55]:
df_giro['position'] = df_giro['position'].apply(lambda x:int(x.split('.')[0]))

In [56]:
df_giro['name'] = df_giro['name'].apply(lambda x:' '.join(x.split(' ')[1:]))

In [57]:
df_giro['time'] = df_giro['time'].apply(fix_time)

In [58]:
stage = []
for n in range(len(df_giro)//10):
    stage = stage + [n+1]*10

In [59]:
df_giro['stage'] = stage

In [60]:
names = df_giro.loc[
    (
        (df_giro['stage']==df_giro['stage'].max()) &
        (df_giro['position']<=5)
    ) |
    (
        (df_giro['position']==1)
    )
]['name'].unique()

In [61]:
df_giro = df_giro.loc[df_giro['name'].isin(names)]

In [62]:
df_giro = df_giro[
    [
        'name',
        'time',
        'stage'
    ]
]

In [63]:
time_max = df_giro['time'].max()

In [64]:
time_max = (1+time_max//100)*100

In [65]:
time_max

200

In [66]:
df_giro = df_giro.pivot(
    columns=['name'],
    index='stage'
)['time']

In [67]:
df_giro = df_giro.fillna(1000)

In [68]:
df_index = (pd.Series(list(range(1,140+1,1)), name='stage')/10).to_frame()

In [69]:
df_giro = df_giro.reset_index()

In [70]:
df_giro['stage'] = df_giro['stage'].astype(float)

In [71]:
#df_giro = pd.merge(df_index, df_giro, on='stage', how='left')

In [72]:
#df_giro.interpolate(method='polynomial', order=5)

In [73]:
df_giro = df_giro.melt(
    id_vars = 'stage',
    var_name = 'name',
    value_name = 'time'
)

In [74]:
df_giro

,stage,name,time
0,1.0,Aleksandr Vlasov,1000.0
1,2.0,Aleksandr Vlasov,1000.0
2,3.0,Aleksandr Vlasov,27.0
3,4.0,Aleksandr Vlasov,84.0
4,5.0,Aleksandr Vlasov,84.0
...,...,...,...
107,10.0,Simon Yates,56.0
108,11.0,Simon Yates,82.0
109,12.0,Simon Yates,82.0
110,13.0,Simon Yates,82.0


In [75]:
fig = px.line(df_giro, x='stage', y='time', color='name')
fig.update_layout(yaxis_range=[time_max, -1])
fig.show()